## 1: Load data from a csv file

In [1]:
import unicodecsv

# File names
enrollments_filename = 'enrollments.csv'
engagement_filename = 'daily_engagement.csv'
submissions_filename = 'project_submissions.csv'

# Function: Read .csv file into a list of dicts
# Returns a list of dict entries (dataset rows)
def read_csv(filename):
    with open(filename, 'rb') as f: # Open file for reading
        reader = unicodecsv.DictReader(f) # Read each row as a dict
        return list(reader) # Return here because exiting with closes the file

# List of entries for each dataset
enrollments = read_csv(enrollments_filename)
daily_engagement = read_csv(engagement_filename)
project_submissions = read_csv(submissions_filename)

# Check first entries
print(enrollments[0])
print(daily_engagement[0])
print(project_submissions[0])

{u'status': u'canceled', u'is_udacity': u'True', u'is_canceled': u'True', u'join_date': u'2014-11-10', u'account_key': u'448', u'cancel_date': u'2015-01-14', u'days_to_cancel': u'65'}
{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


## 2: Fix data types

In [2]:
from datetime import datetime as dt

# Takes a date as a string and returns a Python datetime object. 
# Returns None if there is no date given.
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')

# Takes a string (either an empty string or represents an integer).
# Returns an int if non-empty string, None if empty string.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

In [3]:
# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'days_to_cancel': 65,
 u'is_canceled': True,
 u'is_udacity': True,
 u'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 u'status': u'canceled'}

In [4]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed'])) # Convert string to float then int
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{u'acct': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [5]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

## 3: Rename keys in list of dicts

In [6]:
# For each entry (dicts) in daily_engagement, 
# Create 'account_key' key and delete 'acct' key
for entry in daily_engagement:
    entry['account_key'] = entry['acct']
    del[entry['acct']]

# Show updated key
daily_engagement[0]

{'account_key': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

## 4: Investigate the data

1. Find the total number of rows in each table
2. Find the total number of unique students (account keys) in each table

In [10]:
# Takes a list of dictionaries
# Returns a set of the unique students in the list input
def get_unique_students(data_list):
    unique_students = set() # Create empty set
    for entry in data_list:
        unique_students.add(entry['account_key'])
    return unique_students

# Show unique students in each table
print("Unique students:")
print("Enrollments", len(get_unique_students(enrollments)))
print("Daily engagement", len(get_unique_students(daily_engagement)))
print("Project submissions", len(get_unique_students(project_submissions)))

# Show total number of rows in each table
print("\nTotal rows:")
print("Enrollments", len(enrollments))
print("Daily engagement", len(daily_engagement))
print("Project submissions", len(project_submissions))

Unique students:
('Enrollments', 1302)
('Daily engagement', 1237)
('Project submissions', 743)

Total rows:
('Enrollments', 1640)
('Daily engagement', 136240)
('Project submissions', 3642)


## 5.1: Identify problem cases in the data
- Student records in enrollment data with no corresponding engagement record

In [22]:
# Identify enrollment records with no corresponding engagement record

# Get set of unique students in engagement
unique_studs_engagement = get_unique_students(daily_engagement)

# New list for no-engagement studs
no_engagement_studs = []

# For each entry in enrollments, 
# get entries not in set of students in engagements data
for entry in enrollments:
    student = entry['account_key']
    if student not in unique_studs_engagement:
        no_engagement_studs.append(entry)

# Print number of studs with no engagement
print(len(no_engagement_studs))

# Print example of students with no engagement record
print(no_engagement_studs[0])

71
{u'status': u'canceled', u'is_udacity': False, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 12, 0, 0), u'account_key': u'1219', u'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), u'days_to_cancel': 0}


In [27]:
# Get student enrollment records for which days_to_cancel is not zero

# New list for non-zero days_to_cancel students
no_engagement_studs_weird = list()

# For each entry in no_engagement_studs,
# get entries for which days_to_cancel is not zero
for entry in no_engagement_studs:
    if entry['days_to_cancel'] != 0:
        no_engagement_studs_weird.append(entry)

# Print number of studs with weird case
print(len(no_engagement_studs_weird))

# Print example of students with weird case
print(no_engagement_studs_weird[0])

3
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}


## 5.2: Identify problem cases in the data
1. Identify entries in enrollment data that are Udacity test accounts
2. Clear all Udacity test accounts from datasets

In [28]:
# Set of account_key-s for Udacity test accounts
udacity_test_acct_set = set()

# For each enrollments entry, check if udacity test account
for entry in enrollments:
    if entry['is_udacity']:
        udacity_test_acct_set.add(entry['account_key'])

# Show udacity test accounts
udacity_test_acct_set

{u'1069', u'1101', u'1304', u'312', u'448', u'818'}

In [29]:
# Function: Takes in a dataset (list of dicts)
# Returns dataset with Udacity accounts removed

def remove_udacity_accounts(dataset):
    non_udacity_data = [] # New list for non-udacity data
    for entry in dataset:
        if entry['account_key'] not in udacity_test_acct_set:
            non_udacity_data.append(entry)
    return non_udacity_data

non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagements = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagements))
print(len(non_udacity_submissions))

1622
135656
3634
